## 4. Trend Analysis of Programming Languages (2013-2019)

This separate notebook is for trend analysis and wrangling techniques neccessary to parse responses stored in a variety of formats.

Similarly to the previous notebook, we will import our packages and get the dataset and schema file names.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from IPython import display
import seaborn as sns
from collections import defaultdict, Counter
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [2]:
datasets = glob.glob("datasets/*_public.csv")
datasets

['datasets\\2013_survey_results_public.csv',
 'datasets\\2014_survey_results_public.csv',
 'datasets\\2015_survey_results_public.csv',
 'datasets\\2016_survey_results_public.csv',
 'datasets\\2017_survey_results_public.csv',
 'datasets\\2018_survey_results_public.csv']

In [3]:
schemas = glob.glob("datasets/*_schema.csv")
schemas

['datasets\\2017_survey_results_schema.csv',
 'datasets\\2018_survey_results_schema.csv']

In [4]:
results = {}
results

{}

### 2013 Dataset

Earlier datasets do not have traditional headers, instead the first row of these datasets are the questions and the second row are the possible responses of the question, such as C, C#, PHP etc.
Therefore, instead of parsing multilevel indexes, it would be simpler to get the columns by ``idx`` - the function defined below allows this.

In [5]:
def get_index(column, column_value, schema):
    '''
    INPUT - column - string - the name of the column we want to search
            column_value - string - the term we would like to search for
    OUTPUT - 
            idx - index - the integer index of the column containing the specified value
    '''
    idx = schema.index[schema[column].astype(str).str.contains(str(column_value))]
    return idx

We can create somewhat of a schema by parsing the first two rows of the 2013 dataset to reference as a schema and fetch data by index. The columns in the schema will be defined as ``Question`` and ``Option``. 

In [7]:
def make_schema_load_dataset(data_path):
    '''
    INPUT - datapath - string - the location of the file we want to load
    OUTPUT - 
            schema, dataset - tuple(DataFrame) - two pandas dataframes, one of the schema and another, the data.
    '''
    schema = pd.read_csv(data_path, index_col=None, header=None, nrows=2, encoding="utf-8", low_memory=False).T
    schema.columns = ['Question', 'Option']
    dataset = pd.read_csv(data_path, index_col=None, header=1, encoding="utf-8", low_memory=False)
    return schema, dataset

In [8]:
df_2013_schema, df_2013_data = make_schema_load_dataset(datasets[0])

In [9]:
df_2013_schema.head(3)

,Question,Option
0,What Country or Region do you live in?,Response
1,Which US State or Territory do you live in?,Response
2,How old are you?,Response


We obtain the index by passing the value from the dataset to find, which is string matched to our values in the schema.

In [10]:
index = get_index('Question', 'Which of the following languages or technologies have you used significantly in the past year?', df_2013_schema) 

We can then slice the dataframe into a subset containing only the desired language information.

In [11]:
df_2013_schema.iloc[index[0]:index[0]+14]

,Question,Option
56,Which of the following languages or technologi...,C
57,NaN,C++
58,NaN,C#
59,NaN,Java
60,NaN,JavaScript
61,NaN,jQuery
62,NaN,JQuery
63,NaN,Node.js
64,NaN,Objective-C
65,NaN,PHP


In [12]:
df_2013_languages = df_2013_data.iloc[:, index[0]:index[0]+14]
df_2013_languages.head(3)

,C,C++,C#,Java,JavaScript,jQuery,JQuery,Node.js,Objective-C,PHP,Python,Ruby,SQL,Other (please specify)
0,NaN,NaN,NaN,Java,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SQL,NaN
1,NaN,NaN,C#,NaN,JavaScript,jQuery,NaN,NaN,NaN,PHP,NaN,NaN,NaN,MySql / VbScript
2,NaN,NaN,C#,NaN,JavaScript,jQuery,NaN,NaN,NaN,NaN,NaN,NaN,SQL,"PL/SQL, XSLT, XQuery"


In [13]:
df_2013_languages.shape

(9742, 14)

We can retrieved a dictionary of counts and calculate the percentages using the ``percentage_dict`` function defined below, which can also remove users who didn't report any languages. The results are stored in the results dictionary, with the key ``2013``.

In [14]:
language_dict_2013 = df_2013_languages.describe().loc['count'].to_dict()
language_dict_2013

{'C': 1436,
 'C++': 1696,
 'C#': 3027,
 'Java': 3020,
 'JavaScript': 4735,
 'jQuery': 4109,
 'JQuery': 239,
 'Node.js': 599,
 'Objective-C': 955,
 'PHP': 2324,
 'Python': 1880,
 'Ruby': 794,
 'SQL': 4593,
 'Other (please specify)': 1897}

In [15]:
def percentage_dict(languages, dataframe, rows=None):
    '''
    INPUT - languages - dict - the dictionary of language counts
            dataframe - DataFrame - the dataframe we would like to process
            rows - boolean - the rows we want to remove nulls from
    OUTPUT - 
            idx - index - the integer index of the column containing the specified value
    '''
    print("Rows Before: {}".format(dataframe.shape[0]))
    if rows is not None:
        dataframe = dataframe[rows].dropna(how='all', axis=0)
    else:
        dataframe = dataframe.dropna(how='all', axis=0)
        
    n = dataframe.shape[0]
    print("Rows After: {}".format(n))
    return dict((k, v/n) for k, v in languages.items())

In [16]:
results['2013'] = percentage_dict(language_dict_2013, df_2013_languages)

Rows Before: 9742
Rows After: 8042


In [17]:
results

{'2013': {'C': 0.1785625466301915,
  'C++': 0.21089281273315097,
  'C#': 0.3763989057448396,
  'Java': 0.3755284755036061,
  'JavaScript': 0.5887838846058194,
  'jQuery': 0.5109425516040785,
  'JQuery': 0.02971897537925889,
  'Node.js': 0.07448395921412584,
  'Objective-C': 0.11875155433971649,
  'PHP': 0.28898284008952996,
  'Python': 0.23377269335986073,
  'Ruby': 0.09873165879134543,
  'SQL': 0.5711265854265108,
  'Other (please specify)': 0.235886595374285}}